In [ ]:
import json
import requests
import pandas as pd
from io import StringIO
from confluent_kafka import Producer

In [ ]:
#fetchs the crypto data from crypto compare with API request
def fetch_crypto_data(api_key, fsym, tsym, limit):
    """
    Fetches historical hourly data for a specific cryptocurrency pair over a defined limit of hours.
    """
    url = 'https://min-api.cryptocompare.com/data/v2/histohour'
    parameters = {
        'fsym': fsym,
        'tsym': tsym,
        'limit': limit,
        'api_key': api_key
    }
    response = requests.get(url, params=parameters)
    print(response)
    data = response.json()['Data']['Data']
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['coin'] = fsym  # Add a column with the cryptocurrency symbol
   
    return df

In [31]:
# Funaction for mentain a data files of the crypto
def manage_data_file(coin, api_key):
    filename = f"{coin}_hourly_data.csv"
    base_currency = 'USD'

    # Fatch missing data for crypto
    if not os.path.exists(filename):
        end_date = datetime.datetime.now()
        limit = 2000

        print(f"Fatech new data for {coin}")

        df = fetch_crypto_data(api_key, coin, base_currency, limit)
        df.to_csv(filename, index=False)

    # In case the file is already exists, fatch the missing data
    else:
        existing_df = pd.read_csv(filename)
        existing_df['time'] = pd.to_datetime(existing_df['time'])

        last_date = existing_df['time'].max()
        end_date = datetime.datetime.now()

        if last_date > end_date:
            print(f"Data is updated for {coin}")
            return existing_df

        print(f" Fatch data of {coin} form {last_date} to {end_date}")
        limit = (end_date - last_date)
        limit = limit.total_seconds()//3600
        print(limit)
        df = fetch_crypto_data(api_key, coin, base_currency, limit)

        # concatinate new data with the exist one.
        full_df = pd.concat([existing_df, df]).drop_duplicates(subset=['time'])
        full_df.to_csv(filename, index=False)

    return pd.read_csv(filename)


In [ ]:
#first preprocessing the data
def preprocess_data(df):
    # Drop unnecessary columns
    df.drop(["conversionType", "conversionSymbol","coin"], axis = 'columns', inplace = True)

    # Dealing with time columns
    df['time'] = pd.to_datetime(df['time'], errors='coerce')
    df = df.set_index('time')

    return df

In [ ]:
#sending the data to the consumer
def send_data(coin_name, data):

    producer.produce('crypto_topic',key = coin_name, value=data)
    producer.flush()
    print(f"Successfully sent data for {coin_name}")


In [ ]:
#User Interface
producer = Producer({'bootstrap.servers': 'localhost:9092'})

topic = 'bt'
batch_size = 200
API_KEY = 'ecb769259e45868df9bd2de303bea33c7259f00239195768c20899e88a05e2d3'
limit = 168  #Fetch previous data for one week before
base_currency = 'USD'  # Convert to USD

while True:
    coin = input("Enter the coin symbol you like to invest in:\n to exit just hit enter ")
    if coin is None or coin == "":
        producer.produce("exit", value="exit")
        producer.flush()
        print("Exiting the program.")
        break
    try:
        df = manage_data_file(coin, API_KEY)
    except Exception as e:
        print(f"Error fetching data for {coin}: {e}")
        continue
    df = preprocess_data(df)
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    csv_data = csv_buffer.getvalue()
    # raw_data = fetch_crypto_data(API_KEY, coin, base_currency, limit)
    send_data(coin, csv_data)

producer.produce("crypto_topic", key="exit", value="")